# Setup

In [1]:
import sys
import importlib

src_path = '/Users/jblandin/my/school-local/cs511/pywumpuslite'

def reload_module_by_name(mod_name, var_name, path):
    if path is not None and path not in sys.path:
        sys.path.append(path)
        print('Appended {} to sys.path'.format(path))
    else:
        print('{} already in sys.path'.format(path))
                
    for mod in list(sys.modules.keys()):
        if mod_name in mod:
            del sys.modules[mod]
    
    if var_name in globals():
        del globals()[var_name] # deletes the variable named <var_name>
    return importlib.__import__(mod_name)

def reload(path=src_path):
    global agent_function, agent, environment, simulation, transfer_percept, world_application, action
    agent_function = reload_module_by_name('agent_function', 'agent_function', path=path)
    agent = reload_module_by_name('agent', 'agent', path=path)
    environment = reload_module_by_name('environment', 'environment', path=path)
    simulation = reload_module_by_name('simulation', 'simulation', path=path)
    transfer_percept = reload_module_by_name('transfer_percept', 'transfer_percept', path=path)
    world_application = reload_module_by_name('world_application', 'world_application', path=path)
    action = reload_module_by_name('action', 'action', path=path)
    import agent_function, agent, environment, simulation, transfer_percept, world_application
    
reload(src_path)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path


# Run

In [20]:
reload()
print()
out_writer = open('wumpus_output.txt', 'w')

world_application.run_application(num_trials=1, max_steps=50)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path

Dimensions: 4 x 4
Maximum number of steps: 50
Number of trials: 1
Random agent location: False
Random number seed: 993030
Output filename: wumpus_output.txt
Non-Deterministic Behavior: False

Posterior Death probs
---------------------

 0.191  0.191  0.191  0.191 

 0.191  0.191  0.191  0.191 

 0.191  0.191  0.191  0.191 

 -----  0.191  0.191  0.191 


 -----------------------
|     | P W |     |     | 
|     |     |     |     | 
 -----------------------
|     | P   |     |     | 
| G   |     |